# Linear Algebra: Matrix Transpose & Differential Calculus

In [1]:
import numpy as np
np.set_printoptions(suppress=True)

In [2]:
x = np.array([1,1,4,5,6,8,9])
x

array([1, 1, 4, 5, 6, 8, 9])

In [3]:
# 『신경망 첫걸음, 2016』 에서는 이 방식으로 변환한다.
np.array(x, ndmin=2).T

array([[1],
       [1],
       [4],
       [5],
       [6],
       [8],
       [9]])

In [4]:
x.T

array([1, 1, 4, 5, 6, 8, 9])

In [5]:
x.shape[0]

7

In [6]:
# 그러나 reshape로 좀 더 우아하게 변환할 수 있다.
x = x.reshape(-1, 1)
x

array([[1],
       [1],
       [4],
       [5],
       [6],
       [8],
       [9]])

In [7]:
# 엉뚱한 답을 찾으라고 할 순 없으므로 미리 정답 행렬로 답을 만들어 둔다.
# w = np.array([[2,5,1,7,2,5,1], [1,4,5,1,2,4,1], [2,4,2,2,2,2,2]])

# 우리가 학습하고자 하는 가중치 행렬
def init_w():
    w = np.array([[1,5,3,7,4,5,66], [2,4,23,1,2,1,1], [4,4,1,1,1,1,2]],  dtype=np.float)
    return w

w = init_w()
w

array([[ 1.,  5.,  3.,  7.,  4.,  5., 66.],
       [ 2.,  4., 23.,  1.,  2.,  1.,  1.],
       [ 4.,  4.,  1.,  1.,  1.,  1.,  2.]])

In [8]:
np.dot(x, w)

ValueError: shapes (7,1) and (3,7) not aligned: 1 (dim 1) != 3 (dim 0)

(3, 7)과 (7, 1) 처럼 첫 번째 matrix의 열과 두 번째 matrix의 행이 일치해야 점곱(dot product)이 가능하다.

In [9]:
y = np.dot(w, x)
y

array([[711.],
       [132.],
       [ 49.]])

In [10]:
# 정답
t = np.array([[107],[83],[70]])
t

array([[107],
       [ 83],
       [ 70]])

In [11]:
y - t # E

array([[604.],
       [ 49.],
       [-21.]])

In [12]:
# 미분

# d(E) / d(w_kj) = d() / d(w_kj) * sum((t_n - y_n) ** 2)

# 그러나 결과 y_n은 연결되는 노드로부터만 영향을 받는다.
# 다시 말해, 노드 k의 결과 값인 y_k는 그와 연결되는 가중치 w_kj에 의해서만 영향을 받는다.

# 따라서 w_kj의 연결 노드인 y_k 외에 모든 y_n을 제거할 수 있다. 이제 sum을 제거할 수 있다.
# d(E) / d(w_kj) = d((t_k - y_k) ** 2) / d(w_kj)

# 미분의 연쇄 법칙
# d(E) / d(w_kj) = d(E) / d(y_k) * d(y_k) / d(w_kj)

# d(E) / d(y_k) = -2 * (t_k - y_k)
# d(y_k) / d(w_kj) = d(sum(w_kn * x_n)) / d(w_kj)

# 마찬가지로 y_k는 이와 연결되는 w_kj에 의해서만 영향을 받는다.
# 따라서 w_kj의 연결 노드인 w_kj * x_j 에만 영향을 주므로 sum을 제거할 수 있다.
# d(w_kj * x_j) / d(w_kj) = x_j

# 미분 계산을 쉽게 하기 위해 1/2 * SE를 취한다.
# d(1/2 * (t_k - y_k) ** 2) / d(y_k) = y_k - t_k

# 모두 정리하면 아래와 같은 깔끔한 수식이 된다.
# (y_k - t_k) * x_j

In [13]:
l = 0.001 # learning rate, 0.01로 했을 경우 overshooting이 발생했다.

for k in range(3):
    for j in range(7):
        delta_w = (y[k] - t[k]) * x[j] # 미분
        delta_w = -1 * l * delta_w     # 미분의 역방향 * learning rate
        # print (k, j, t[k], y[k], x[j], delta_w)
        w[k][j] += delta_w

In [14]:
# 역전파 1회 완료 후 계산된 가중치 행렬
w

array([[ 0.396,  4.396,  0.584,  3.98 ,  0.376,  0.168, 60.564],
       [ 1.951,  3.951, 22.804,  0.755,  1.706,  0.608,  0.559],
       [ 4.021,  4.021,  1.084,  1.105,  1.126,  1.168,  2.189]])

In [15]:
# 다시 가중치 행렬 초기화
w = init_w()
w

array([[ 1.,  5.,  3.,  7.,  4.,  5., 66.],
       [ 2.,  4., 23.,  1.,  2.,  1.,  1.],
       [ 4.,  4.,  1.,  1.,  1.,  1.,  2.]])

In [16]:
# 이번에는 전체 계산을 한 번에 진행한다.
# [[d(E_1) / d(y_1)],[d(E_2) / d(y_2)],[d(E_3) / d(y_3)]] 
# \dot [x_1, x_2, x_3 ... x_7]

# 미분 계산 편의상 1/2을 취한다.
# d(1/2 * E_1) / d(y_1) = 1/2 * (t_1 - y_1) ** 2
# = y_1 - t_1

# x_n 과 dot 계산 결과는 [w_11, w_12, w_13 ... w_17], [w_21, w_22 ... w_27], [w_31 ...] 와 동일하다.
# 에러를 x_n의 비율만큼 나눠준다.

# 아래와 같이 미분과 전치 행렬로 한 번에 동일한 계산 결과를 줄 수 있다.
w += - l * np.dot((y - t), x.transpose())

In [17]:
w

array([[ 0.396,  4.396,  0.584,  3.98 ,  0.376,  0.168, 60.564],
       [ 1.951,  3.951, 22.804,  0.755,  1.706,  0.608,  0.559],
       [ 4.021,  4.021,  1.084,  1.105,  1.126,  1.168,  2.189]])

In [18]:
y-t, x.transpose()

(array([[604.],
        [ 49.],
        [-21.]]), array([[1, 1, 4, 5, 6, 8, 9]]))

In [19]:
np.dot((y - t), x.transpose())

array([[ 604.,  604., 2416., 3020., 3624., 4832., 5436.],
       [  49.,   49.,  196.,  245.,  294.,  392.,  441.],
       [ -21.,  -21.,  -84., -105., -126., -168., -189.]])

In [20]:
# 추가 학습
for _ in range(50):
    w += - l * np.dot((y - t), x.transpose())
    y = np.dot(w, x)
y

array([[107.00133721],
       [ 83.00010848],
       [ 69.99995351]])

In [21]:
w

array([[ -1.6964226 ,   2.3035774 ,  -7.78569041,  -6.48211301,
        -12.17853561, -16.57138081,  41.73219658],
       [  1.78125048,   3.78125048,  22.12500194,  -0.09374758,
          0.68750291,  -0.74999613,  -0.96874564],
       [  4.09374979,   4.09374979,   1.37499917,   1.46874896,
          1.56249875,   1.74999834,   2.84374813]])

아래는 미분의 계산 일치 여부를 직접 검증한다.

In [22]:
import sympy
sympy.init_printing(use_latex='mathjax')
t, y = sympy.symbols('t y')
e = (t - y)**2
e

       2
(t - y) 

In [23]:
sympy.Derivative(e, y)

∂ ⎛       2⎞
──⎝(t - y) ⎠
∂y          

In [24]:
sympy.Derivative(e, y).doit()

-2⋅t + 2⋅y

In [25]:
e12 = 1/2 * (t - y) ** 2
sympy.Derivative(e12, y)

∂ ⎛           2⎞
──⎝0.5⋅(t - y) ⎠
∂y              

In [26]:
sympy.Derivative(e12, y).doit()

-1.0⋅t + 1.0⋅y